In [ ]:
!pip install pymongo pymatgen matminer scikit-learn smact pandas atomate fireworks

In [ ]:
!pip install mp-api

In [ ]:
!pip install --upgrade mp-api

In [ ]:
!apt-get update
!apt-get install -y texlive texlive-latex-extra texlive-fonts-recommended dvipng cm-super

In [ ]:
!pip install dask[dataframe]

In [ ]:
!pip install dask[complete] dask-ml matminer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
### Imports ###
import os
import pandas as pd
import smact
from smact import Species  # Import Species class from smact library for representing species
from smact.oxidation_states import Oxidation_state_probability_finder  # Import Oxidation_state_probability_finder class from smact library for finding oxidation states
from pymatgen.core.composition import Composition  # Import Composition class from pymatgen library for representing compositions
from pymatgen.core.structure import Structure  # Import Structure class from pymatgen library for representing crystal structures
from pymatgen.analysis.structure_prediction.substitution_probability import SubstitutionPredictor
from pymatgen.ext.matproj import MPRester
from pymatgen.analysis.bond_valence import BVAnalyzer
from pymatgen.io.cif import CifWriter
import json

In [ ]:
useful_BGs_used = pd.read_csv('drive/MyDrive/data/Bandgaps/filtered_useful_BGs.csv')
# Rename the column
useful_BGs_used = useful_BGs_used.rename(columns={'pretty_formula': 'formula_pretty'})
# Display the first few rows to confirm the change
print(useful_BGs_used.head())
useful_BGs_used.describe()

   Unnamed: 0 formula_pretty composition_obj  0-norm    2-norm    3-norm  \
0         165        LiBeFeO  Li1 Be1 Fe1 O1     4.0  0.500000  0.396850   
1         173     LiBe(CoO)2  Li1 Be1 Co2 O2     4.0  0.527046  0.436790   
2         183       LiBeCoO4  Li1 Be1 Co1 O4     4.0  0.622700  0.580221   
3         298    LiBe(MoO2)2  Li1 Be1 Mo2 O4     4.0  0.586302  0.524792   
4         302       Li2BeMoO  Li2 Be1 Mo1 O1     4.0  0.529150  0.444796   

     5-norm    7-norm   10-norm  minimum Number  ...  avg ionic char  \
0  0.329877  0.304753  0.287175             3.0  ...        0.131551   
1  0.385263  0.368439  0.357293             3.0  ...        0.140157   
2  0.571763  0.571444  0.571429             3.0  ...        0.154361   
3  0.503277  0.500565  0.500049             3.0  ...        0.140263   
4  0.407234  0.401326  0.400117             3.0  ...        0.132654   

   band center  HOMO_character  HOMO_element  HOMO_energy  LUMO_character  \
0    -1.764143               d   

,Unnamed: 0,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,minimum Number,maximum Number,range Number,...,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char,band center,HOMO_energy,LUMO_energy,gap_AO,gbr_gap
count,1.783300e+04,17833.0,17833.000000,17833.000000,17833.000000,17833.000000,17833.000000,17833.000000,17833.000000,17833.000000,...,17833.000000,17833.000000,17833.000000,17833.000000,17833.000000,17833.000000,17833.000000,17833.000000,17833.000000,17833.000000
mean,1.716694e+06,4.0,0.588358,0.525421,0.498867,0.492881,0.489838,7.702350,65.008075,57.305725,...,2.840999,2.518925,0.146358,0.642850,0.115284,-2.321422,-0.225943,-0.222739,0.003204,1.750080
std,9.367078e+05,0.0,0.061514,0.085514,0.103250,0.109118,0.112770,1.035162,16.504300,16.425818,...,1.583990,2.445449,0.353475,0.124394,0.027984,0.370841,0.052302,0.053096,0.013211,0.011513
min,1.650000e+02,4.0,0.500000,0.396850,0.329877,0.304753,0.287175,3.000000,20.000000,12.000000,...,0.000000,0.000000,0.000000,0.183314,0.022018,-3.070597,-0.338381,-0.338381,0.000000,1.730002
25%,8.607210e+05,4.0,0.529150,0.444796,0.407234,0.401326,0.400117,8.000000,50.000000,43.000000,...,1.583333,0.000000,0.000000,0.563121,0.096728,-2.600631,-0.266540,-0.266540,0.000000,1.740014
50%,1.831704e+06,4.0,0.577350,0.517872,0.501229,0.500098,0.500003,8.000000,73.000000,65.000000,...,2.600000,2.000000,0.000000,0.681744,0.115808,-2.376573,-0.220603,-0.217760,0.000000,1.750122
75%,2.478602e+06,4.0,0.648074,0.609120,0.600524,0.600040,0.600001,8.000000,78.000000,70.000000,...,3.750000,4.000000,0.000000,0.733532,0.135219,-2.067298,-0.182464,-0.180198,0.000000,1.760020
max,3.216891e+06,4.0,0.744123,0.728690,0.727286,0.727273,0.727273,8.000000,83.000000,80.000000,...,9.090909,12.600000,1.000000,0.908097,0.202455,-1.003085,-0.085375,-0.078699,0.132508,1.769999


In [ ]:
# There are some elements we definitely don't want from the get-go
unwanted_els = ['Be','Hg','Pb','Tl','Pr','Nd','Sm','Gd','Dy','Ho','Er','Tm','Lu','Hf','Ta']

# Convert to dict for ease
all_comps = list(useful_BGs_used.T.to_dict().values())

# reduce down to wanted compounds
wanted_comps = []
for i in all_comps:
    list_els = Composition(i['composition_obj']).elements
    wanted = True
    for el in list_els:
        if el.symbol in unwanted_els:
            wanted = False
    if wanted == True:
        wanted_comps.append(i)

# Work out sustainability score (based on HHI) for each composition
def sus_calc(comp):
    sus_factor = 0
    for i in comp.elements:
        sus_factor += (comp.get_wt_fraction(i) *smact.Element(i.symbol).HHI_r)
    return sus_factor

for i in wanted_comps:
    i['sus_factor'] = sus_calc(Composition(i['composition_obj']))

# Return to dataframe
filtered_useful_BGs = pd.DataFrame.from_dict(wanted_comps)

In [ ]:
filtered_useful_BGs = filtered_useful_BGs.sort_values(by='sus_factor', ascending=True)
filtered_useful_BGs = filtered_useful_BGs.reset_index(drop=True)
selected_formulas = list(filtered_useful_BGs['formula_pretty'])

In [ ]:
import concurrent.futures

# Replace 'YOUR_API_KEY' with your actual API key
api_key = "XXXX"
mpr = MPRester(api_key)

def fetch_and_decorate(formula):
    try:
        entries = mpr.get_entries(formula)
        decorated_structures = []
        for entry in entries:
            structure = entry.structure
            try:
                bva = BVAnalyzer()
                structure = bva.get_oxi_state_decorated_structure(structure)
                decorated_structures.append(structure)
            except Exception as e:
                print(f"Oxidation states could not be added for {formula}: {e}")
        return decorated_structures
    except Exception as e:
        print(f"Failed to fetch structure for {formula}: {e}")
        return []

# Generator to yield batches of formulas to process
def formula_batches(formulas, batch_size):
    for i in range(0, len(formulas), batch_size):
        yield formulas[i:i + batch_size]

# List to hold all decorated structures
all_decorated_structures = []

batch_size = 10  # Adjust batch size based on available memory
max_workers = 4  # Adjust based on available CPU cores

# Process formulas in batches to manage memory usage
for batch in formula_batches(selected_formulas, batch_size):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = executor.map(fetch_and_decorate, batch)
        for result in results:
            all_decorated_structures.extend(result)

print(f"Total structures fetched and decorated: {len(all_decorated_structures)}")

Retrieving ThermoDoc documents: 0it [00:00, ?it/s]

Retrieving ThermoDoc documents: 0it [00:00, ?it/s]

Retrieving ThermoDoc documents: 0it [00:00, ?it/s]

Retrieving ThermoDoc documents: 0it [00:00, ?it/s]

Retrieving ThermoDoc documents: 0it [00:00, ?it/s]

...

Retrieving ThermoDoc documents: 0it [00:00, ?it/s]

Retrieving ThermoDoc documents: 0it [00:00, ?it/s]

Retrieving ThermoDoc documents: 0it [00:00, ?it/s]

Retrieving ThermoDoc documents: 0it [00:00, ?it/s]

Total structures fetched and decorated: 111


In [ ]:
from pymatgen.analysis.structure_prediction.substitutor import Substitutor

# Initialize the substitution predictor with a probability threshold
substitutor = Substitutor(threshold=1e-5)

# List to hold new predicted structures
predicted_structures = []

# Iterate through the fetched structures
for structure in all_decorated_structures:
    try:
        substitutions = substitutor.pred_from_structures([structure], remove_duplicates=True, remove_existing=False)
        predicted_structures.extend(substitutions)
    except Exception as e:
        print(f"Substitution prediction failed for a structure: {e}")

print(f"Total predicted structures: {len(predicted_structures)}")

In [ ]:
import os
import json

# Ensure the directory exists
save_dir = 'drive/MyDrive/data/Structures'
os.makedirs(save_dir, exist_ok=True)

# Path to save the JSON file
save_path = os.path.join(save_dir, 'predicted_structures.json')

# Convert predicted structures to a list of dictionaries
predicted_structures_dicts = [s.as_dict() for s in predicted_structures]

# Save to a JSON file
with open(save_path, 'w') as f:
    json.dump(predicted_structures_dicts, f, indent=4)

print(f"Predicted structures saved to {save_path}")

In [ ]:
with open('drive/MyDrive/data/Structures/predicted_structures.json', 'r') as f:
    tetra_element_oxide_for_calc = json.load(f)

# Convert back to pymatgen Structure objects
tetra_element_oxide_for_calc = [Structure.from_dict(i) for i in tetra_element_oxide_for_calc]

In [ ]:
# Initialize an instance of the Oxidation_state_probability_finder class
ox = Oxidation_state_probability_finder()

# Create a list of metals and oxygen to consider
metals_and_anions = smact.metals + ['O']

# Initialize a counter for the number of compounds that pass the test
num_passes = 0

# Iterate over each structure in the list of tetra-element oxides
for struc in tetra_element_oxide_for_calc:
    # Get a list of pymatgen species in the structure
    species = [i.specie for i in struc]
    # Filter the species list to include only metals and oxygen
    species = [i for i in species if i.symbol in metals_and_anions]

    # Pass the species to the probability calculator and get the probability
    prob = ox.compound_probability(species)

    # Check if the probability is below the threshold
    if prob < 0.005:
        print(species)
        print('Below threshold!')
    else:
        # Increment the counter if the probability is above the threshold
        num_passes += 1

# Print the number of compounds that passed the oxidation state probability test
print('number of compounds to pass the oxidation state probability test: {}'.format(num_passes))